### 분류(Classification) 
- classification 알고리즘은 예측하려는 대상의 변수를 입력 받고
목표가 갖고 있는 카테고리 값 중에 어느 한 값으로 분류하여 예측
- 지도 학습 유형에 속함 

### KNN (k-Nearest-Neighbors)
- 새로운 관측값이 주어지면 기존 데이터 중에 가장 속성이 비슷한 k개의 이웃 찾음
- 가까운 이웃들이 갖고 있는 목표 값과 같은 값으로 분류하여 예측

In [1]:
# Step-1 : 데이터 준비

import pandas as pd 
import seaborn as sns

df = sns.load_dataset('titanic')

df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
pd.set_option('display.max_columns', 15)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


- info() 메소드로 데이터와 자료형 개수 확인
- 누락 데이터가 포함되있으면 데이터의 특성과 분석 목표에 맞춰 누락 데이터 처리

In [3]:
# Step-2 : 데이터 탐색

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


- NaN 값이 많은 deck 열 삭제
- embarked와 내용이 겹치는 embark_town 열 삭제

In [4]:
rdf = df.drop(['deck', 'embark_town'], axis = 1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

- 'age' 열의 누락 데이터가 177개 포함되있는 것을 확인
- 분석에 포함시켜야하는 속성이므로 누락 데이터가 있는 행 제거

In [5]:
rdf = rdf.dropna(subset = ['age'], how = 'any', axis = 0)
len(rdf)

714

In [6]:
most_freq = rdf['embarked'].value_counts(dropna = True).idxmax()
most_freq

'S'

In [7]:
rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# embarked 열의 NaN는 'S'로 바꿈 
rdf['embarked'].fillna(most_freq, inplace = True)

- 변수로 사용할 후보 열 선택
- 예측 변수로 생존 여부를 나타내는 'survived' 열 추가
- head() 메소드로 데이터 구성 확인

In [9]:
# Step-3 : 속성 선택
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


- KNN 모형에 적용하기 위해 'sex'와 'embarked' 열의 범주형 데이터를 숫자형으로 변환 
- 이를 One-hot-encoding이라 함 

In [10]:
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis = 1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [11]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [12]:
ndf.drop(['sex', 'embarked'], axis = 1, inplace = True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


- 예측 변수인 'survived' 열을 변수 y에 저장 
- 나머지 열들을 설명 변수로 사용하기 위해 변수 X에 할당
- sklearn의 preprocessing 모듈 중 train_test_split() 메소드를 사용
    - 훈련 데이터와 검증 데이터를 나눔(검증 30%)

In [20]:
# Step-4 : 훈련/검증 데이터 분할

X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C', 'town_Q', 'town_S']]
y = ndf['survived']

X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  714 non-null    int64  
 1   age     714 non-null    float64
 2   sibsp   714 non-null    int64  
 3   parch   714 non-null    int64  
 4   female  714 non-null    uint8  
 5   male    714 non-null    uint8  
 6   town_C  714 non-null    uint8  
 7   town_Q  714 non-null    uint8  
 8   town_S  714 non-null    uint8  
dtypes: float64(1), int64(3), uint8(5)
memory usage: 31.4 KB


In [19]:
from sklearn import preprocessing 
X = preprocessing.StandardScaler().fit(X).transform(X)

X.shape

(714, 9)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)
print('train data 개수 : ', X_train.shape)
print('test data 개수 : ', X_test.shape)

train data 개수 :  (499, 9)
test data 개수 :  (215, 9)


- sklearn 라이브러리의 neighbors 모듈 사용 
- KNeighborsClassifier() 함수로 KNN 분류 모형 객체 생성 
- predict() 메소드에 전달하여 예측값 확인

In [23]:
# Step-5 : 모형 학습 및 검증 

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

y_hat = knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])


[0 0 1 0 0 1 0 1 0 0]
[0 0 1 0 0 1 1 1 0 0]


- 모형의 예측 능력 평가 
- metrics 모듈의 confusion_matrix() 함수 사용

In [24]:
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

array([[111,  14],
       [ 29,  61]], dtype=int64)

In [26]:
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       125
           1       0.81      0.68      0.74        90

    accuracy                           0.80       215
   macro avg       0.80      0.78      0.79       215
weighted avg       0.80      0.80      0.80       215

